In [137]:
import os
import json
import requests
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [138]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_API_URL = os.getenv("GOOGLE_API_URL")

In [131]:
from dataclasses import dataclass, asdict
from datetime import datetime


@dataclass
class Video:
    id: str
    title: str
    url: str
    description: str
    published_at: datetime


@dataclass
class Channel:
    id: str
    name: str
    videos: list[Video] = None

    def __post_init__(self):
        self.videos = []

    def add_video(self, video: Video):
        self.videos.append(video)

In [132]:
channells = [
    {"name": "ByteByteGo", "id": "UCZgt6AzoyjslHTC9dz0UoTw"},
    {"name": "IBM Technology", "id": "UCKWaEZ-_VweaEx1j62do_vQ"},
    {"name": "Sebastian Rashka", "id": "UC_CzsS7UTjcxJ-xXp1ftxtA"},
    {"name": "FreeCodeCamp.org", "id": "UC8butISFwT-Wl7EV0hUK0BQ"},
]

channel_list = [Channel(**channel) for channel in channells]

In [133]:
channel_list

[Channel(id='UCZgt6AzoyjslHTC9dz0UoTw', name='ByteByteGo', videos=[]),
 Channel(id='UCKWaEZ-_VweaEx1j62do_vQ', name='IBM Technology', videos=[]),
 Channel(id='UC_CzsS7UTjcxJ-xXp1ftxtA', name='Sebastian Rashka', videos=[]),
 Channel(id='UC8butISFwT-Wl7EV0hUK0BQ', name='FreeCodeCamp.org', videos=[])]

In [ ]:
for channel in channel_list:
    skip = False

    params = {
        "key": GOOGLE_API_KEY,
        "channelId": channel.id,
        "part": ["snippet", "id"],
        "order": "date",
        "maxResults": 50,
        "pageToken": None,
    }

    while not skip:
        response = requests.get(GOOGLE_API_URL, params=params)
        data = response.json()

        if "items" in data:
            for item in data["items"]:
                if item["id"]["kind"] == "youtube#video":
                    video_id = item["id"]["videoId"]
                    video_title = item["snippet"]["title"]
                    video_description = item["snippet"]["description"]
                    video_published_at = item["snippet"]["publishTime"]
                    video_url = f"https://www.youtube.com/watch?v={video_id}"

                    try:
                        video_published_at = datetime.strptime(
                            item["snippet"]["publishTime"], "%Y-%m-%dT%H:%M:%SZ"
                        )
                    except ValueError:
                        video_published_at = None

                    video = Video(
                        id=video_id,
                        title=video_title,
                        url=video_url,
                        description=video_description,
                        published_at=video_published_at,
                    )

                    channel.add_video(video)

        if "nextPageToken" not in data:
            skip = True
        else:
            params["pageToken"] = data.get("nextPageToken")

In [135]:
channel_list

[Channel(id='UCZgt6AzoyjslHTC9dz0UoTw', name='ByteByteGo', videos=[Video(id='wvLdBJEl-wc', title='Why is Kafka FAST? Part 1', url='https://www.youtube.com/watch?v=wvLdBJEl-wc', description='', published_at=datetime.datetime(2025, 5, 8, 14, 45, 59)), Video(id='2g1G8Jr88xU', title='System Design Was HARD - Until You Knew the Trade-Offs, Part 2', url='https://www.youtube.com/watch?v=2g1G8Jr88xU', description='Sign up now to access ChatLLM: https://bit.ly/42RlGDV Get a Free System Design PDF with 158 pages by subscribing to our ...', published_at=datetime.datetime(2025, 4, 30, 15, 30, 3)), Video(id='1nENigGr-a0', title='System Design Was HARD - Until You Knew the Trade-Offs', url='https://www.youtube.com/watch?v=1nENigGr-a0', description='Get a Free System Design PDF with 158 pages by subscribing to our weekly newsletter: https://bit.ly/bbg-social Animation tools: ...', published_at=datetime.datetime(2025, 4, 22, 15, 30)), Video(id='f7KgDNZU3-Y', title='gRPC, WebSocket and WebHook!', url='